In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
seed = 123
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
rio_de_janeiro_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/data/samples/sample_rio_de_janeiro_edital_capital.json', lines=True)    

In [7]:
rio_de_janeiro_sample = rio_de_janeiro_df.sample(1, random_state=seed)['conteudo'].to_list()
rio_de_janeiro_sample

['Proc. 0039512-74.2008.8.19.0205 (2008.205.039484-0) - IZABEL CRISTINA MATOS RUBENS, IGOR MATOS DE MORAES (Adv(s). Dr(a). LEONARDO ORSINI DE CASTRO AMARANTE (OAB/RJ-055328) X TRANSPORTES ORIENTAL LTDA Despacho: Diante do certificado as fls.295, remetam-se os autos ao arquivo.']

In [8]:
rio_de_janeiro_sample_tokenized = tok.texts_to_sequences(rio_de_janeiro_sample)
pd.DataFrame(rio_de_janeiro_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,1342,1,848,280,15,479,15327,280,3808,1,...,1733,29,31,3923,1238,8,21,34,19,387


In [9]:
rio_de_janeiro_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=rio_de_janeiro_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(rio_de_janeiro_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1342,1,848,280,15,479,15327,280,3808,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predicted_rio_de_janeiro_sample_tokenized_padded = model(rio_de_janeiro_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_rio_de_janeiro_sample_tokenized_padded[0]):
    if (i < len(rio_de_janeiro_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[rio_de_janeiro_sample_tokenized[0][i]])

1 Proc
1 UNK
1 74
1 2008
1 8
1 19
1 0205
14 2008
1 205
1 UNK
1 0
1 IZABEL
1 CRISTINA
1 MATOS
1 RUBENS
1 IGOR
1 MATOS
1 DE
1 MORAES
1 Adv
1 s
1 Dr
1 a
1 LEONARDO
1 ORSINI
1 DE
1 CASTRO
1 AMARANTE
1 OAB
1 RJ
1 UNK
1 X
1 TRANSPORTES
1 UNK
1 LTDA
1 Despacho
1 Diante
1 do
1 certificado
1 as
1 fls
1 295
1 remetam
1 se
1 os
1 autos
1 ao
1 arquivo
